In [1]:
import numpy as np
import os
import gzip
import struct
import random
import time

import scipy.ndimage as nd
import PIL.Image
from IPython.display import clear_output, Image, display
from io import BytesIO

In [2]:
def loadlabels(filepath):
    with gzip.open(filepath, "rb") as f:
        _magic, n = struct.unpack(">ll", f.read(2*4))
        assert(_magic == 2049)
        return f.read(n)

def loadimages(filepath):
    with gzip.open(filepath, "rb") as f:
        _magic, n, sx, sy = struct.unpack(">llll", f.read(4*4))
        assert(_magic == 2051)
        images = []
        for _ in range(n):
            images.append(np.frombuffer(f.read(sx*sy), dtype=np.dtype("B")).astype(np.float32)/255.0)
        return ((sx, sy), images)

def load(labelsfp, imagesfp):
    labels = loadlabels(labelsfp)
    images = loadimages(imagesfp)
    assert(len(labels) == len(images[1]))
    return (images[0], list(zip(labels, images[1])))

def mkimg(data, size, fmt='jpeg'):
    data = np.uint8(np.clip((1 - data.reshape(size))*255, 0, 255))
    f = BytesIO()
    PIL.Image.fromarray(data).save(f, fmt)
    return Image(data=f.getvalue())

In [3]:
print("loading train set")
imgsize, trainset = load("data/train-labels-idx1-ubyte.gz", "data/train-images-idx3-ubyte.gz")

print("loading test set")
_imgsize, testset = load("data/t10k-labels-idx1-ubyte.gz", "data/t10k-images-idx3-ubyte.gz")

assert(imgsize == _imgsize)

loading train set
loading test set


In [4]:
indices = np.random.randint(0, len(trainset),  5)
for digit, image in [(trainset[i][0], mkimg(trainset[i][1], imgsize)) for i in indices]:
    display(digit, image)

4

6

2

4

7

In [5]:
class Net:
    def __init__(self, sizes, mag=0):
        self.sizes = sizes
        if mag == 0:
            self.weights = [np.zeros((sx, sy), dtype=np.float32) for sx, sy in zip(sizes[1:], sizes[:-1])]
            self.biases = [np.zeros(s, dtype=np.float32) for s in sizes]
        else:
            self.weights = [mag*np.random.randn(sx, sy) for sx, sy in zip(sizes[1:], sizes[:-1])]
            self.biases = [mag*np.random.randn(s) for s in sizes]

    def __iter__(self):
        return iter(self.weights + self.biases)

    def zcopy(self):
        return Net(self.sizes)

    def mkmem(self):
        return [[np.zeros_like(b) for _ in range(2)] for b in self.biases]

    def fill(self, value):
        for w in self.weights:
            w.fill(value)
        for b in self.biases:
            b.fill(value)

In [6]:
def act(x):
    return np.tanh(x)

def act_deriv(x):
        return 1/np.cosh(x)**2

def cost(a, y):
    d = a - y
    return np.dot(d, d)/2

def cost_deriv(a, y):
    return a - y

In [7]:
def feedforward(net, x, fmem=False):
    a = x + net.biases[0]
    if fmem:
        mem = net.mkmem()
        mem[0][0] = a
    else:
        mem = None

    for i, (w, b) in enumerate(zip(net.weights, net.biases[1:])):
        a = act(a)
        if fmem:
            mem[i][1] = a

        a = np.dot(w, a) + b
        if fmem:
            mem[i + 1][0] = a

    return (a, mem)

In [8]:
def backprop(net, a, y, mem, neterr, rate):
    e = cost_deriv(a, y)
    zmem = list(zip(*mem))
    zlist = list(zip(
        net.weights, net.biases[1:],
        neterr.weights, neterr.biases[1:],
        zmem[0][1:], zmem[1][:-1]
    ))
    last = True
    for w, b, ew, eb, v, ap in reversed(zlist):
        if not last:
            e = e*act_deriv(v)
        eb += e*rate
        ew += np.outer(e, ap)*rate
        e = np.dot(e.transpose(), w)
        last = False
    neterr.biases[0] += e

In [ ]:
net = Net((imgsize[0]*imgsize[1], 15, 10), mag=1e-2)
neterr = net.zcopy()

In [16]:
batchsize = 10
rate = 1e-2
for iepoch in range(10):
    print("epoch %d:" % iepoch)
#    print("train:")
    totalcost = 0.0
    tstart = time.time()
    random.shuffle(trainset)
    for batch in [trainset[p:p+batchsize] for p in range(0, len(trainset), batchsize)]:
        for digit, img in batch:
            res = np.array([i == digit for i in range(10)])
            out, netmem = feedforward(net, img, fmem=True)
            totalcost += cost(out, res)
            backprop(net, out, res, netmem, neterr, rate)
        for v, ev in zip(net, neterr):
            v -= ev/batchsize
        neterr.fill(0)
#    print("cost avg: %.4f" % (totalcost/len(testset)))
#    print("time elapsed: %.2f s" % (time.time() - tstart))

#    print("test:")
    totalcost = 0.0
    hitcount = 0
    tstart = time.time()
    for digit, img in testset:
        res = np.array([i == digit for i in range(10)])
        out, _ = feedforward(net, img)
        totalcost += cost(out, res)
        hitcount += digit == np.argmax(out)
#    print("cost avg: %.4f" % (totalcost/len(testset)))
    print("hit count: %d / %d" % (hitcount, len(testset)))
#    print("time elapsed: %.2f s" % (time.time() - tstart))

print("done")

epoch 0:
hit count: 9231 / 10000
epoch 1:
hit count: 9227 / 10000
epoch 2:
hit count: 9196 / 10000
epoch 3:
hit count: 9133 / 10000
epoch 4:
hit count: 9208 / 10000
epoch 5:
hit count: 9206 / 10000
epoch 6:
hit count: 9183 / 10000
epoch 7:
hit count: 9207 / 10000
epoch 8:
hit count: 9160 / 10000
epoch 9:
hit count: 9193 / 10000
done


In [19]:
random.shuffle(testset)
for digit, img in testset[:5]:
        out, _ = feedforward(net, img)
        display(digit, np.argmax(out), mkimg(img, imgsize))

0

0

8

8

5

5

8

8

2

2

In [34]:
rate = 1e-1
for digit in range(10):
    img = np.random.random(net.sizes[0])*0.1
    res = np.array([i == digit for i in range(10)])
    for i in range(10000):
        out, mem = feedforward(net, img, fmem=True)
        neterr.fill(0)
        cv = cost(out, res)
        backprop(net, out, res, mem, neterr, rate)
        img -= neterr.biases[0]
        img = np.clip(img, 0, 1)
    display(np.argmax(out), cv, mkimg(img, imgsize))

0

7.225406595495336e-06

1

7.470851848325101e-13

2

...

8.655734943382941e-09

8

2.929495455570694e-07

9

2.586579520428043e-14